In [1]:
#train3 from 2: use adam, remove weight_decay; one BW channel (not 3)
import torch,os 
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import v2
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt

# from models import *
from utils import progress_bar
from tqdm import tqdm
from configparser import ConfigParser
from torch.utils.data import  DataLoader
from LIDC_data import LIDC_Dataset

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

prep_tr = [
    v2.Lambda(lambda x: torch.clamp(x,-1000,400)),
    v2.Lambda(lambda x: (x+1000)/1400),
    v2.CenterCrop((384,384)),
    # v2.Lambda(lambda x: x.expand(3,-1,-1))
]
aug_tr = [
    v2.RandomAffine(degrees=10),
    v2.RandomHorizontalFlip(),
    v2.GaussianNoise(0,0.1)
]
trans_train = v2.Compose( prep_tr + aug_tr )
trans_test = v2.Compose( prep_tr  )

In [3]:

parser = ConfigParser()
parser.read('.settings')
root_dir = parser.get('dataset','root_dir') #/workspaces/data/lidc-idri/slices
meta_dir = parser.get('dataset','meta_dir') #/workspaces/data/lidc-idri/splits
train_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'train_malB.csv'),transform=trans_train)
test_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'test_malB.csv'),transform=trans_test)
total_train_data = len(train_data)
total_test_data = len(test_data)
print('total_train_data:',total_train_data, 'total_test_data:',total_test_data)

trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16)

total_train_data: 5495 total_test_data: 2354


In [4]:
# net = ResNet18(num_classes=2)
net = models.resnet18(num_classes=2)
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# net.fc = nn.Linear(net.fc.in_features, 2)
net = net.to(device)

In [5]:
lr = 1e-4
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

trainning_accuracy=[]
trainning_loss=[]
testing_accuracy=[]
testing_loss=[]

In [6]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader,desc=f"[Epoch {epoch+1}]")):
        inputs, targets = inputs.to(device), targets.to(device)
        # targets = targets.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss = criterion(outputs, targets)
        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        # correct += (preds == targets.long()).sum().item()

    train_acc = 100.*correct/total
    train_loss = train_loss/(batch_idx+1)
    print(f"Train Loss: {train_loss}, Train Acc: {train_acc:.2f}%")
    trainning_accuracy.append(train_acc)
    trainning_loss.append( train_loss )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            # targets = targets.float().unsqueeze(1)
            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # correct += (preds == targets.long()).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_acc = 100.*correct/total
        test_loss = test_loss/(batch_idx+1)
        print(f"Test Loss: {test_loss}, Test Acc: {test_acc:.2f}%")
        testing_accuracy.append(100.*correct/total)
        testing_loss.append(test_loss/(batch_idx+1))
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_owndata.pth')
        best_acc = acc

In [ ]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
 
 #   scheduler.step()

[Epoch 1]: 100%|██████████| 344/344 [01:21<00:00,  4.23it/s]


Train Loss: 0.6958181294937467, Train Acc: 54.09%
 [=================================================>]  Step: 33ms | Tot: 17s324ms | Loss: 0.679 | Acc: 55.565% (1308/2354 148/148  
Test Loss: 0.6787409359538877, Test Acc: 55.56%
Saving..


[Epoch 2]: 100%|██████████| 344/344 [01:20<00:00,  4.26it/s]


Train Loss: 0.6791260236397733, Train Acc: 57.11%
 [=================================================>]  Step: 19ms | Tot: 17s464ms | Loss: 0.846 | Acc: 56.500% (1330/2354 148/148  
Test Loss: 0.8462995912476972, Test Acc: 56.50%
Saving..


[Epoch 3]: 100%|██████████| 344/344 [01:21<00:00,  4.24it/s]


...

[Epoch 49]: 100%|██████████| 344/344 [01:19<00:00,  4.34it/s]


Train Loss: 0.15487157474380248, Train Acc: 94.49%
 [=================================================>]  Step: 20ms | Tot: 16s919ms | Loss: 0.703 | Acc: 76.381% (1798/2354 148/148  
Test Loss: 0.7033789543485319, Test Acc: 76.38%


[Epoch 50]: 100%|██████████| 344/344 [01:19<00:00,  4.34it/s]


Train Loss: 0.15642357612743454, Train Acc: 94.38%
 [=================================================>]  Step: 21ms | Tot: 17s84ms | Loss: 0.938 | Acc: 70.858% (1668/2354 148/148   
Test Loss: 0.937515304146989, Test Acc: 70.86%


: 